In [1]:
import pandas as pd
import numpy as np
import os
import os.path
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score
import nltk
from nltk.tokenize import word_tokenize

### 1. Préparation de l'ensemble des données

##### Chargement des données

In [2]:
# Liste des fichiers pour chaque discipline
Liste_fichiers_sport = {}
Liste_repertoires = os.listdir('/home/ali_mjyw_leking/bbcsport')[0:4] + [os.listdir('/home/ali_mjyw_leking/bbcsport')[-1]]
#print("Nombre de textes par classe: ")
#print("\n")

# Création d'un dataframe et chargement des données dans le dataframe

df = pd.DataFrame()

etiquettes = {'football' : 0, 'cricket' : 1, 'tennis': 2, 'athletics': 3, 'rugby': 4}

for dossier in Liste_repertoires:
    
    path = '/home/ali_mjyw_leking/bbcsport/'+dossier
    
    liste_files = os.listdir(path)
    
    for file in liste_files:
        
            with open(os.path.join(path, file),'r', encoding='utf-8') as infile:
                txt = infile.read()
            df = df.append([[txt, etiquettes[dossier], dossier]],ignore_index=True)

In [3]:
df.columns = ['Textes', 'likelihood', 'Nom_discipline']
df

Textes  likelihood  \
0    Stuart joins Norwich from Addicks\n\nNorwich h...           0   
1    Gunners clock up winning record\n\nArsenal cla...           0   
2    Mourinho sends out warning shot\n\nChelsea bos...           0   
3    Boro suffer Morrison injury blow\n\nMiddlesbro...           0   
4    Spain coach faces racism inquiry\n\nSpain's Fo...           0   
..                                                 ...         ...   
732  Wilkinson to miss Ireland match\n\nEngland wil...           4   
733  Scots suffer another injury blow\n\nScotland's...           4   
734  England coach faces rap after row\n\nEngland c...           4   
735  Wood - Ireland can win Grand Slam\n\nFormer ca...           4   
736  Moore questions captaincy\n\nBrian Moore belie...           4   

    Nom_discipline  
0         football  
1         football  
2         football  
3         football  
4         football  
..             ...  
732          rugby  
733          rugby  
734          rugby  
735          rugby  
736          rugby  

[737 rows x 3 columns]

In [4]:
#pip install texthero

### Nettoyage des textes avec la librairie texthero

In [5]:
import texthero as hero

In [6]:
df['Textes_clean'] = hero.clean(df['Textes'])

In [7]:
df

Textes  likelihood  \
0    Stuart joins Norwich from Addicks\n\nNorwich h...           0   
1    Gunners clock up winning record\n\nArsenal cla...           0   
2    Mourinho sends out warning shot\n\nChelsea bos...           0   
3    Boro suffer Morrison injury blow\n\nMiddlesbro...           0   
4    Spain coach faces racism inquiry\n\nSpain's Fo...           0   
..                                                 ...         ...   
732  Wilkinson to miss Ireland match\n\nEngland wil...           4   
733  Scots suffer another injury blow\n\nScotland's...           4   
734  England coach faces rap after row\n\nEngland c...           4   
735  Wood - Ireland can win Grand Slam\n\nFormer ca...           4   
736  Moore questions captaincy\n\nBrian Moore belie...           4   

    Nom_discipline                                       Textes_clean  
0         football  stuart joins norwich addicks norwich signed ch...  
1         football  gunners clock winning record arsenal claimed p...  
2         football  mourinho sends warning shot chelsea boss jose ...  
3         football  boro suffer morrison injury blow middlesbrough...  
4         football  spain coach faces racism inquiry spain footbal...  
..             ...                                                ...  
732          rugby  wilkinson miss ireland match england take irel...  
733          rugby  scots suffer another injury blow scotland back...  
734          rugby  england coach faces rap row england coach andy...  
735          rugby  wood ireland win grand slam former captain kei...  
736          rugby  moore questions captaincy brian moore believes...  

[737 rows x 4 columns]

#### Autres nettoyages

In [8]:
from texthero import preprocessing

In [9]:
custom_pipeline = [preprocessing.fillna
                   , preprocessing.lowercase
                   , preprocessing.remove_digits
                   , preprocessing.remove_punctuation
                   , preprocessing.remove_diacritics
                   , preprocessing.remove_stopwords
                   , preprocessing.remove_whitespace
                   , preprocessing.stem]

In [10]:
df['Textes_clean'] = hero.clean(df['Textes_clean'], pipeline = custom_pipeline)
df.head()

Textes  likelihood  \
0  Stuart joins Norwich from Addicks\n\nNorwich h...           0   
1  Gunners clock up winning record\n\nArsenal cla...           0   
2  Mourinho sends out warning shot\n\nChelsea bos...           0   
3  Boro suffer Morrison injury blow\n\nMiddlesbro...           0   
4  Spain coach faces racism inquiry\n\nSpain's Fo...           0   

  Nom_discipline                                       Textes_clean  
0       football  stuart join norwich addick norwich sign charlt...  
1       football  gunner clock win record arsenal claim premiers...  
2       football  mourinho send warn shot chelsea boss jose mour...  
3       football  boro suffer morrison injuri blow middlesbrough...  
4       football  spain coach face racism inquiri spain footbal ...

#### Inspecter des principaux mots afin de voir d'éventuels autres mots vides:

In [11]:
tw = hero.visualization.top_words(df['Textes_clean']).head(10)
import plotly.express as px
fig = px.bar(tw)
fig.show()
tw.head()

said     1338
play     1009
game      937
year      886
first     792
Name: Textes_clean, dtype: int64

On voit bien qu'il n'y a pas de mots vides

In [12]:
# Détermination du tfidf de chaque document

df['tfidf_clean_text'] = hero.tfidf(df['Textes_clean'])

In [13]:
#df['tfidf_clean_text'] = df['tfidf_clean_text'].apply(lambda x: x)
#df['tfidf_clean_text'] = pd.to_numeric(df['tfidf_clean_text'],errors = 'coerce')
#df['tfidf_clean_text'] = df['tfidf_clean_text'].astype(float, errors = 'raise')
# roundplaces = np.round(df['DATA ENTRY'], decimals = 2)

df.info()
 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 737 entries, 0 to 736
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Textes            737 non-null    object
 1   likelihood        737 non-null    int64 
 2   Nom_discipline    737 non-null    object
 3   Textes_clean      737 non-null    object
 4   tfidf_clean_text  737 non-null    object
dtypes: int64(1), object(4)
memory usage: 28.9+ KB


In [14]:
#df['tfidf_clean_text'].apply(lambda x: x*100)

In [15]:
df.head()

Textes  likelihood  \
0  Stuart joins Norwich from Addicks\n\nNorwich h...           0   
1  Gunners clock up winning record\n\nArsenal cla...           0   
2  Mourinho sends out warning shot\n\nChelsea bos...           0   
3  Boro suffer Morrison injury blow\n\nMiddlesbro...           0   
4  Spain coach faces racism inquiry\n\nSpain's Fo...           0   

  Nom_discipline                                       Textes_clean  \
0       football  stuart join norwich addick norwich sign charlt...   
1       football  gunner clock win record arsenal claim premiers...   
2       football  mourinho send warn shot chelsea boss jose mour...   
3       football  boro suffer morrison injuri blow middlesbrough...   
4       football  spain coach face racism inquiri spain footbal ...   

                                    tfidf_clean_text  
0  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...  
1  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...  
2  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...  
3  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...  
4  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...

In [16]:
#df['tfidf_clean_text'][0]

In [17]:
Df = df[["Textes_clean", "likelihood", "Nom_discipline"]]
Df

Textes_clean  likelihood  \
0    stuart join norwich addick norwich sign charlt...           0   
1    gunner clock win record arsenal claim premiers...           0   
2    mourinho send warn shot chelsea boss jose mour...           0   
3    boro suffer morrison injuri blow middlesbrough...           0   
4    spain coach face racism inquiri spain footbal ...           0   
..                                                 ...         ...   
732  wilkinson miss ireland match england take irel...           4   
733  scot suffer anoth injuri blow scotland back ro...           4   
734  england coach face rap row england coach andi ...           4   
735  wood ireland win grand slam former captain kei...           4   
736  moor question captainci brian moor believ engl...           4   

    Nom_discipline  
0         football  
1         football  
2         football  
3         football  
4         football  
..             ...  
732          rugby  
733          rugby  
734          rugby  
735          rugby  
736          rugby  

[737 rows x 3 columns]

Divisons l'ensemble de données en ensembles d'entraînement et de validation afin de pouvoir entraîner et tester le classificateur. De plus, nous encoderons notre colonne cible afin qu'elle puisse être utilisée dans des modèles d'apprentissage automatique.

In [18]:
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble
import pandas, xgboost, numpy, textblob, string

In [19]:
#pip install -U textblob

In [20]:
#pip install xgboost


Procédons à la division de l'ensemble des données en ensembles d'entraînement et de validation afin de pouvoir entraîner et tester le classificateur. 
De plus, nous encoderons notre colonne cible afin qu'elle puisse être utilisée dans des modèles d'apprentissage automatique.

In [21]:
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(Df['Textes_clean'], Df['Nom_discipline'])

# label encode the target variable 
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)

### Étape du Feature Engineering

In [22]:
# créer un objet de vectorisation de comptage 
count_vect  =  CountVectorizer(analyzer = 'word', token_pattern = r'\w{1,}') 
count_vect.fit(Df['Textes_clean'])

# Transformation des données d'apprentissage et de validation à l'aide de l'objet de vectorisation de comptage

xtrain_count  =  count_vect.transform(train_x) 
xvalid_count  =  count_vect.transform(valid_x)

### Vecteurs TF-IDF en tant que caractéristiques

In [23]:
# word level tf-idf
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000)
tfidf_vect.fit(Df['Textes_clean'])
xtrain_tfidf =  tfidf_vect.transform(train_x)
xvalid_tfidf =  tfidf_vect.transform(valid_x)

# ngram level tf-idf 
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram.fit(Df['Textes_clean'])
xtrain_tfidf_ngram =  tfidf_vect_ngram.transform(train_x)
xvalid_tfidf_ngram =  tfidf_vect_ngram.transform(valid_x)

# characters level tf-idf
tfidf_vect_ngram_chars = TfidfVectorizer(analyzer='char', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram_chars.fit(Df['Textes_clean'])
xtrain_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(train_x) 
xvalid_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(valid_x)

/home/ali_mjyw_leking/anaconda3/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:506: UserWarning:

The parameter 'token_pattern' will not be used since 'analyzer' != 'word'



### Word Embeddings

In [24]:
#pip install keras

In [25]:
#pip install tensorflow

In [26]:
#from keras.preprocessing.sequence import pad_sequences
from keras.utils import pad_sequences
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers

2022-08-03 23:08:00.875617: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-08-03 23:08:00.878416: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-03 23:08:00.878424: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [27]:
# # chargemment des vecteurs d'incorporation de mots pré-formés 
embeddings_index = {}
for i, line in enumerate(open('wiki-news-300d-1M.vec')):
    values = line.split()
    embeddings_index[values[0]] = numpy.asarray(values[1:], dtype='float32')

# création  d'un tokenizer 

token = text.Tokenizer()
token.fit_on_texts(Df['Textes_clean'])
word_index = token.word_index

# convertion des textes en séquence de jetons et les complète pour garantir des vecteurs de longueur égale 
train_seq_x = pad_sequences(token.texts_to_sequences(train_x), maxlen=70)
valid_seq_x = pad_sequences(token.texts_to_sequences(valid_x), maxlen=70)

# création d'un mappage d'intégration de jetons 
embedding_matrix = numpy.zeros((len(word_index) + 1, 300))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

#### Fonctionnalités basées sur le texte / NLP

In [28]:
Df['char_count'] = Df['Textes_clean'].apply(len)
Df['word_count'] = Df['Textes_clean'].apply(lambda x: len(x.split()))
Df['word_density'] = Df['char_count'] / (Df['word_count']+1)
Df['punctuation_count'] = Df['Textes_clean'].apply(lambda x: len("".join(_ for _ in x if _ in string.punctuation))) 
Df['title_word_count'] = Df['Textes_clean'].apply(lambda x: len([wrd for wrd in x.split() if wrd.istitle()]))
Df['upper_case_word_count'] = Df['Textes_clean'].apply(lambda x: len([wrd for wrd in x.split() if wrd.isupper()]))

/tmp/ipykernel_4998/1183645396.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [29]:
Df.head()

Textes_clean  likelihood  \
0  stuart join norwich addick norwich sign charlt...           0   
1  gunner clock win record arsenal claim premiers...           0   
2  mourinho send warn shot chelsea boss jose mour...           0   
3  boro suffer morrison injuri blow middlesbrough...           0   
4  spain coach face racism inquiri spain footbal ...           0   

  Nom_discipline  char_count  word_count  word_density  punctuation_count  \
0       football        1000         158      6.289308                  0   
1       football        3431         528      6.485822                  0   
2       football         706         114      6.139130                  0   
3       football         470          73      6.351351                  0   
4       football         926         145      6.342466                  0   

   title_word_count  upper_case_word_count  
0                 0                      0  
1                 0                      0  
2                 0                      0  
3                 0                      0  
4                 0                      0

In [30]:
pos_family = {
    'noun' : ['NN','NNS','NNP','NNPS'],
    'pron' : ['PRP','PRP$','WP','WP$'],
    'verb' : ['VB','VBD','VBG','VBN','VBP','VBZ'],
    'adj' :  ['JJ','JJR','JJS'],
    'adv' : ['RB','RBR','RBS','WRB']
}

# fonction permettant de vérifier et d'obtenir la partie du nombre de balises vocales d'un mot dans une phrase donnée
def check_pos_tag(x, flag):
    cnt = 0
    try:
        wiki = textblob.TextBlob(x)
        for tup in wiki.tags:
            ppo = list(tup)[1]
            if ppo in pos_family[flag]:
                cnt += 1
    except:
        pass
    return cnt

Df['noun_count'] = Df['Textes_clean'].apply(lambda x: check_pos_tag(x, 'noun'))
Df['verb_count'] = Df['Textes_clean'].apply(lambda x: check_pos_tag(x, 'verb'))
Df['adj_count'] =  Df['Textes_clean'].apply(lambda x: check_pos_tag(x, 'adj'))
Df['adv_count'] =  Df['Textes_clean'].apply(lambda x: check_pos_tag(x, 'adv'))
Df['pron_count'] = Df['Textes_clean'].apply(lambda x: check_pos_tag(x, 'pron'))

####  Modélisation thématique

In [31]:
# train a LDA Model
lda_model = decomposition.LatentDirichletAllocation(n_components=20, learning_method='online', max_iter=20)
X_topics = lda_model.fit_transform(xtrain_count)
topic_word = lda_model.components_ 
vocab = count_vect.get_feature_names()

# view the topic models
n_top_words = 10
topic_summaries = []
for i, topic_dist in enumerate(topic_word):
    topic_words = numpy.array(vocab)[numpy.argsort(topic_dist)][:-(n_top_words+1):-1]
    topic_summaries.append(' '.join(topic_words))

### Construction de modèles

La dernière étape de l'infrastructure de classification de texte consiste à former un classifieur à l'aide des fonctionnalités créées à l'étape précédente. Il existe de nombreux choix de modèles d'apprentissage automatique qui peuvent être utilisés pour former un modèle final. Nous allons implémenter les différents classificateurs suivants à cet effet :

* Decision tree
* Random forest
* Support Vector Machine
* Bagging Models

Implémentons ces modèles et comprenons leurs détails. La fonction suivante est une fonction utilitaire qui peut être utilisée pour former un modèle. Il accepte le classificateur, feature_vector des données d'apprentissage, les étiquettes des données d'apprentissage et les vecteurs de caractéristiques des données valides comme entrées. À l'aide de ces entrées, le modèle est formé et le score de précision est calculé.

In [32]:
def train_model(classifier, feature_vector_train, label, feature_vector_valid, is_neural_net=False):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)
    
    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)
    
    if is_neural_net:
        predictions = predictions.argmax(axis=-1)
    
    return metrics.accuracy_score(predictions, valid_y)

### 1- Decision tree

In [33]:
from sklearn.tree import DecisionTreeClassifier

In [38]:
"""
Pour le calcul de temps d'execution:

import time

l = range(100000)

tps1 = time.clock()
random.shuffle(l)
l.sort()
tps2 = time.clock()
print(tps2 - tps1)

**********************************

import time

start = time.time()

print("The time used to execute this is given below")

end = time.time()

print(end - start)


"""

'\nPour le calcul de temps d\'execution:\n\nimport time\n\nl = range(100000)\n\ntps1 = time.clock()\nrandom.shuffle(l)\nl.sort()\ntps2 = time.clock()\nprint(tps2 - tps1)\n\n**********************************\n\nimport time\n\nstart = time.time()\n\nprint("The time used to execute this is given below")\n\nend = time.time()\n\nprint(end - start)\n\n\n'

In [43]:
# Decision tree on Count Vectors
accu_DT_cv = train_model(DecisionTreeClassifier(random_state = 0), xtrain_count, train_y, xvalid_count)
print("Decision tree, Count Vectors: ", round(accu_DT_cv,2)*100,"%")

# Decision tree on Word Level TF IDF Vectors
accu_DT_wtfidf = train_model(DecisionTreeClassifier(random_state = 0), xtrain_tfidf, train_y, xvalid_tfidf)
print("Decision tree, WordLevel TF-IDF: ",round(accu_DT_wtfidf,2)*100,"%")

# Decision tree on Ngram Level TF IDF Vectors
accu_DT_ngramv = train_model(DecisionTreeClassifier(random_state = 0), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print("Decision tree, N-Gram Vectors: ", round(accu_DT_ngramv,2)*100,"%")

Decision tree, Count Vectors:  94.0 %
Decision tree, WordLevel TF-IDF:  90.0 %
Decision tree, N-Gram Vectors:  87.0 %


### 2- Random forest ou Bagging Models

In [46]:
from sklearn.ensemble import RandomForestClassifier

In [47]:
# Random forest on Count Vectors
accu_Rf_cv = train_model(RandomForestClassifier(n_estimators = 20, random_state = 0), xtrain_count, train_y, xvalid_count)
print("Random forest, Count Vectors: ", round(accu_Rf_cv,2)*100,"%")

# Decision tree on Word Level TF IDF Vectors
accu_Rf_wtfidf = train_model(RandomForestClassifier(n_estimators = 20, random_state = 0), xtrain_tfidf, train_y, xvalid_tfidf)
print("Random forest, WordLevel TF-IDF: ",round(accu_Rf_wtfidf,2)*100,"%")

# Decision tree on Ngram Level TF IDF Vectors
accu_Rf_ngramv = train_model(RandomForestClassifier(n_estimators = 20, random_state = 0), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print("Random forest, N-Gram Vectors: ", round(accu_Rf_ngramv,2)*100,"%")

Random forest, Count Vectors:  95.0 %
Random forest, WordLevel TF-IDF:  95.0 %
Random forest, N-Gram Vectors:  89.0 %


### 3- Support Vector Machine

In [51]:
# Support vector machine on Count Vectors
accu_Svm_cv = train_model(svm.SVC(), xtrain_count, train_y, xvalid_count)
print("Support vector machine, Count Vectors: ", round(accu_Svm_cv,2)*100,"%")

# Decision tree on Word Level TF IDF Vectors
accu_Svm_wtfidf = train_model(svm.SVC(), xtrain_tfidf, train_y, xvalid_tfidf)
print("Support vector machine, WordLevel TF-IDF: ",round(accu_Svm_wtfidf,2)*100,"%")

# SVM sur Ngram Level TF IDF Vectors 
accu_Svm_ngram =  train_model(svm.SVC(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram) 
print("Support vector machine, N-Gram Vectors: ", str(round(accu_Svm_ngram, 2)*100)+"%")

Support vector machine, Count Vectors:  92.0 %
Support vector machine, WordLevel TF-IDF:  93.0 %
Support vector machine, N-Gram Vectors:  88.0%
